In [6]:
# following tensorflow tutorial https://www.tensorflow.org/get_started/get_started
#
import tensorflow as tf
print tf.__version__

# program broken up into nodes consisting a computational tree/graph -- as in MPL!
#
node1 = tf.constant(3.0)
node2 = tf.constant(4.0, dtype=tf.float32)
print node1, node2

# to run it, pass tree/graph to a session which compiles & runs it
#
sess = tf.Session()
print sess.run([node1, node2])

# e.g. summation
#
node3 = tf.add(node1, node2)
print node3
print sess.run([node3])

# parametrized nodes = placeholders
#
a = tf.placeholder(dtype=tf.float32)
b = tf.placeholder(dtype=tf.float32)
s = tf.add(a, b)
print s
print sess.run(s, {a: 3, b: 10})

c = tf.placeholder(dtype=tf.float32)
m = tf.multiply(s, c)
print sess.run(m, {a:4, b: 5, c: 100})

1.3.0
Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)
[3.0, 4.0]
Tensor("Add_2:0", shape=(), dtype=float32)
[7.0]
Tensor("Add_3:0", dtype=float32)
13.0
900.0


In [2]:
# Unlike constants, variable are NOT initialized 
# until global_variables_initializer is called (in the session!)
#
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(dtype=tf.float32)
linear_model = W * x + b # notice shorthand for tf.add and tf.multiply

init = tf.global_variables_initializer()
sess.run(init) # <------------------------------ variables will NOT initialize until this is called!

# run linear model for several values of x simultaneously
#
print sess.run(linear_model, {x: [1, 2, 3, 4]}) 

# to train the model, need target data (y's) and loss f'n
#
y = tf.placeholder(dtype=tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

# can assign (correct) new values
#
fixW = tf.assign(W, [-1.0])
fixb = tf.assign(b, [1])
sess.run([fixW, fixb])
print sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

# gradient descent to find correct values from training examples
#
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init) # reset values to incorrect defaults
for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    
print sess.run([W, b, loss], {x: [1,2,3,4], y: [0,-1,-2,-3]}) 

[ 0.          0.30000001  0.60000002  0.90000004]
23.66
0.0
[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32), 5.6999738e-11]


In [4]:
# a more intense example with an estimator that encapsulates all this logic
#
import numpy as np

feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])

x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

estimator.train(input_fn=input_fn, steps=1000)

train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/var/folders/8r/jkmyb32x3ps7twd729rjlr300000gn/T/tmpoudxxB', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/8r/jkmyb32x3ps7twd729rjlr300000gn/T/tmpoudxxB/model.ckpt.
INFO:tensorflow:loss = 18.0, step = 1
INFO:tensorflow:global_step/sec: 1173.9
INFO:tensorflow:loss = 0.0591029, step = 101 (0.087 sec)
INFO:tensorflow:global_step/sec: 1129.99
INFO:tensorflow:loss = 0.0915569, step = 201 (0.088 sec)
INFO:tensorflow:global_step/sec: 853.934
INFO:tensorflow:loss = 0.0120799, step = 301 (0.117 sec)
INFO:tensorflow:global_step/sec: 1036.62
INFO:tensorflow:loss = 0.00506001, step = 401 (0.096 sec)
INFO:tensorflow:global_step

In [5]:
# manually define your estimator
# features = x
# labels = target y
#
def model_fn(features, labels, mode):
    # linear model
    #
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W * features['x'] + b
    
    # loss and training sub-graphs
    loss = tf.reduce_sum(tf.square(y - labels))
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1))
    
    return tf.estimator.EstimatorSpec(
        mode = mode,
        predictions = y,
        loss = loss,
        train_op = train)
    

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/var/folders/8r/jkmyb32x3ps7twd729rjlr300000gn/T/tmpm4OBaF', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/8r/jkmyb32x3ps7twd729rjlr300000gn/T/tmpm4OBaF/model.ckpt.
INFO:tensorflow:loss = 7.47341230794, step = 1
INFO:tensorflow:global_step/sec: 1254.23
INFO:tensorflow:loss = 0.0274770416402, step = 101 (0.082 sec)
INFO:tensorflow:global_step/sec: 1154.47
INFO:tensorflow:loss = 0.0021579857816, step = 201 (0.087 sec)
INFO:tensorflow:global_step/sec: 1190.66
INFO:tensorflow:loss = 0.000285503910915, step = 301 (0.084 sec)
INFO:tensorflow:global_step/sec: 986.807
INFO:tensorflow:loss = 1.13014331184e-05, step = 401 (0